In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [42]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

loan_bin_encoded = pd.get_dummies(df, columns=["home_ownership", "verification_status", "pymnt_plan", 
                                                  "hardship_flag", "debt_settlement_flag", "initial_list_status", 
                                                 "next_pymnt_d", "application_type"])
loan_bin_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,dti,delinq_2yrs,open_acc,revol_bal,...,inq_last_6mths_1.0,inq_last_6mths_2.0,inq_last_6mths_3.0,inq_last_6mths_4.0,inq_last_6mths_5.0,pub_rec_0.0,pub_rec_1.0,pub_rec_2.0,pub_rec_3.0,pub_rec_4.0
0,10500.0,0.1719,375.35,66000.0,Mar-2019,low_risk,27.24,0.0,8.0,1609.0,...,0,0,0,0,0,1,0,0,0,0
1,25000.0,0.2000,929.09,105000.0,Mar-2019,low_risk,20.23,0.0,17.0,18368.0,...,0,0,0,0,0,0,1,0,0,0
2,20000.0,0.2000,529.88,56000.0,Mar-2019,low_risk,24.26,0.0,8.0,13247.0,...,0,0,0,0,0,1,0,0,0,0
3,10000.0,0.1640,353.55,92000.0,Mar-2019,low_risk,31.44,0.0,10.0,17996.0,...,1,0,0,0,0,0,1,0,0,0
4,22000.0,0.1474,520.39,52000.0,Mar-2019,low_risk,18.76,0.0,14.0,9091.0,...,1,0,0,0,0,1,0,0,0,0


# Split the Data into Training and Testing

In [7]:
# Create our features
X = loan_bin_encoded.drop(columns=["loan_status"])

# Create our target
y = loan_bin_encoded["loan_status"]

In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.148001,1.0,1.0,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.355104,0.0,0.0,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,1.0,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,1.0,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,1.0,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape

(51612, 92)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [27]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification
clf = BalancedRandomForestClassifier(max_depth=100, random_state=0)
clf.fit(X, y)

BalancedRandomForestClassifier(max_depth=100, random_state=0)

In [28]:

# Calculated the balanced accuracy score

from sklearn.metrics import balanced_accuracy_score
y_pred = clf.predict(X_test)

balanced_accuracy_score(y_test, y_pred)

0.9255144995322732

In [29]:
# Display the confusion matrix

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,101,0
Actual 1,2548,14556


In [30]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      1.00      0.85      0.07      0.92      0.86       101
   low_risk       1.00      0.85      1.00      0.92      0.92      0.84     17104

avg / total       0.99      0.85      1.00      0.91      0.92      0.84     17205



In [31]:
#List the feature in a descending order
sorted(zip(clf.feature_importances_, X.columns), reverse=True)

[(0.0772126949782253, 'total_rec_prncp'),
 (0.07300839385736044, 'total_rec_int'),
 (0.058023192640154805, 'total_pymnt'),
 (0.053355509855952284, 'last_pymnt_amnt'),
 (0.048386175810177144, 'total_pymnt_inv'),
 (0.03237429192135993, 'int_rate'),
 (0.024705151780132974, 'dti'),
 (0.018060079127122722, 'out_prncp'),
 (0.017061817104060826, 'bc_util'),
 (0.017025148409504774, 'total_bc_limit'),
 (0.016783705645583593, 'mo_sin_old_rev_tl_op'),
 (0.015807929480872777, 'revol_bal'),
 (0.01571696141515513, 'il_util'),
 (0.015310699536238277, 'tot_hi_cred_lim'),
 (0.015210638674672502, 'avg_cur_bal'),
 (0.014794484688265675, 'max_bal_bc'),
 (0.014760023143133424, 'out_prncp_inv'),
 (0.014592766981376185, 'mths_since_rcnt_il'),
 (0.014558675072593386, 'mo_sin_old_il_acct'),
 (0.014497505384009951, 'installment'),
 (0.013956059466810574, 'mths_since_recent_bc'),
 (0.013941677787318923, 'bc_open_to_buy'),
 (0.013922512051488856, 'mths_since_recent_inq'),
 (0.013861821084310654, 'all_util'),
 (0.

### Easy Ensemble AdaBoost Classifier

In [35]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(random_state=42)
eec.fit(X_train, y_train)
y_predEEC = eec.predict(X_test)

In [36]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_predEEC)

0.7291149542924358

In [43]:
# Display the confusion matrix
print(confusion_matrix(y_test, y_predEEC))

[[   68    33]
 [ 3678 13426]]


In [44]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report(y_test, y_predEEC))

Classification Report
              precision    recall  f1-score   support

   high_risk       0.02      0.67      0.04       101
    low_risk       1.00      0.78      0.88     17104

    accuracy                           0.78     17205
   macro avg       0.51      0.73      0.46     17205
weighted avg       0.99      0.78      0.87     17205

